In [1]:
import json
import numpy as np

In [2]:
labels = {'nose':0,
         'sternum':1,
         'Rshoulder':2,
         'Relbow':3,
         'Rwrist':4,
         'Rhip':5,
         'Rknee':6,
         'Rankle':7,
         'Reye':8,
         'Rear':9,
         'Lshoulder':10,
         'Lelbow':11,
         'Lwrist':12,
         'Lhip':13,
         'Lknee':14,
         'Lankle':15,
         'Leye':16,
         'Lear':17}

val_name_list ={'left_Explorer_HD720_SN15385_19-36-03.png',
               'left_Explorer_HD720_SN15385_19-35-42.png',
               'left_Explorer_HD720_SN15385_19-34-44.png',
               'left_Explorer_HD720_SN15385_19-34-37.png',
               'left_Explorer_HD720_SN15385_19-34-18.png',
               'left_Explorer_HD720_SN15385_19-34-00.png',
               'left_Explorer_HD720_SN15385_19-32-20.png',
               'left_Explorer_HD720_SN15385_19-32-12.png',
               'left_Explorer_HD720_SN15385_19-31-20.png',
               'left_Explorer_HD720_SN15385_19-30-22.png'}

val_flag = 0
sigmas = np.array([.26, 1, .79, .72, .62, 1.07, .87, .89, .25, .35, .79, .72, .62, 1.07, .87, .89, .25, .35])/10.0
variances = (sigmas ** 2)*2

In [3]:
def read_from_gt_dict(gt_dict):
    """
    This function implements reading groundtruth json dict and outputs keypoints and square area
    param: ground truth dict generated by labelme
    """
    res_dict = {}
    
    # recognize how many people in the picture
    flags = gt_dict['flags']
    if flags['1']:
        num = 1
        res_dict['person1'] = {'kpt': np.zeros((2,18)), 'S': 0}
    elif flags['2']:
        num = 2
        res_dict['person1'] = {'kpt': np.zeros((2,18)), 'S': 0}
        res_dict['person2'] = {'kpt': np.zeros((2,18)), 'S': 0}
    elif flags['3']:
        num = 3
        res_dict['person1'] = {'kpt': np.zeros((2,18)), 'S': 0}
        res_dict['person2'] = {'kpt': np.zeros((2,18)), 'S': 0}
        res_dict['person3'] = {'kpt': np.zeros((2,18)), 'S': 0}
    elif flags['4']:
        num = 4
        res_dict['person1'] = {'kpt': np.zeros((2,18)), 'S': 0}
        res_dict['person2'] = {'kpt': np.zeros((2,18)), 'S': 0}
        res_dict['person3'] = {'kpt': np.zeros((2,18)), 'S': 0}
        res_dict['person4'] = {'kpt': np.zeros((2,18)), 'S': 0}
    elif flags['5']:
        num = 5
        res_dict['person1'] = {'kpt': np.zeros((2,18)), 'S': 0}
        res_dict['person2'] = {'kpt': np.zeros((2,18)), 'S': 0}
        res_dict['person3'] = {'kpt': np.zeros((2,18)), 'S': 0}
        res_dict['person4'] = {'kpt': np.zeros((2,18)), 'S': 0}
        res_dict['person5'] = {'kpt': np.zeros((2,18)), 'S': 0}
    elif flags['6']:
        num = 6
        res_dict['person1'] = {'kpt': np.zeros((2,18)), 'S': 0}
        res_dict['person2'] = {'kpt': np.zeros((2,18)), 'S': 0}
        res_dict['person3'] = {'kpt': np.zeros((2,18)), 'S': 0}
        res_dict['person4'] = {'kpt': np.zeros((2,18)), 'S': 0}
        res_dict['person5'] = {'kpt': np.zeros((2,18)), 'S': 0}
        res_dict['person6'] = {'kpt': np.zeros((2,18)), 'S': 0}
    else:
        raise ValueError("Wrong flag")
    
    
    for kpt in gt_dict['shapes']:
        name = kpt['label']
        value = kpt['points']
        person_id = 'person' + str(kpt['group_id'])
        
        if name == 'bbox':
            res_dict[person_id]['S'] = abs((value[0][0] - value[1][0])*(value[0][1] - value[1][1]))
        else:
            idx = labels[name]
            res_dict[person_id]['kpt'][:,idx] = value[0]
    return res_dict

In [4]:
def read_from_pt_dict(pt_dict):
    """
    This function implements reading prediction json dict and outputs keypoints in ndarray format
    param: prediction dict generated by postprocess
    """
    res_dict = {}
    
    for key in pt_dict:
        res_dict[key] = np.zeros((2,18))
        for name in pt_dict[key]:
            idx = labels[name]
            value = pt_dict[key][name]
            res_dict[key][:,idx] = [value[1], value[0]]
    
    return res_dict

In [5]:
def evaluate_oks(pt_dict, gt_dict):
    gt_kpts = []
    gt_area = []
    for person_id in gt_dict:
        gt_kpts.append(gt_dict[person_id]['kpt'])
        gt_area.append(gt_dict[person_id]['S'])
    
    pt_kpts = []
    for person_id in pt_dict:
        pt_kpts.append(pt_dict[person_id])
    
    res_oks = []
    for person_id in range(len(gt_kpts)):
        gt_kpt = gt_kpts[person_id]
        area = gt_area[person_id]
        best_oks = 0
        for pt_kpt in pt_kpts:
            oks = compute_oks(pt_kpt, gt_kpt, area)
            if oks > best_oks:
                best_oks = oks
        
        res_oks.append(best_oks)
            
    return res_oks

In [6]:
def compute_oks(pt_kpt, gt_kpt, area):
    gt_mask = gt_kpt!=0
    
    gt_x = gt_kpt[0,:]
    pt_x = pt_kpt[0,:]
    dx = gt_x - pt_x
    
    gt_y = gt_kpt[1,:]
    pt_y = pt_kpt[1,:]
    dy = gt_y - pt_y
    
    e = (dx**2 + dy**2) /variances/ (area+np.spacing(1)) / 2
    oks = np.exp(-e)
    oks = oks[np.nonzero(gt_mask[0,:])]
    
    return np.sum(oks) / oks.shape[0]

In [7]:
def evaluate_model(pt_json, gt_dir):
    
    with open(pt_json, "r") as f:
        pt_dicts = json.loads(json.load(f))

    n = 1
    oks_all = []
    for img_name in pt_dicts.keys():
        
        if (img_name not in val_name_list) and val_flag:
            continue
            
        print('Processing img.' ,n ,' :', img_name)
        pt_kpts = read_from_pt_dict(pt_dicts[img_name])

        json_name = img_name[:-3] + 'json'
        json_file = gt_dir + json_name
        with open(json_file, "r") as f:
            gt_dict = json.load(f)

        gt_kpts = read_from_gt_dict(gt_dict)

        oks = evaluate_oks(pt_kpts, gt_kpts)
        n += 1

        for each in oks:
            if each < 0.5:
                print(each)
            oks_all.append(each)

    oks_all = np.array(oks_all)
    AP_50 = np.sum(oks_all >= 0.5)/len(oks_all)
    AP_75 = np.sum(oks_all >= 0.75)/len(oks_all)

    T = np.arange(0.5, 1, 0.05)
    mAP = 0
    for t in T:
        temp = np.sum(oks_all >= t)/len(oks_all)
        mAP += temp

    mAP /= T.shape
    mAP
    
    return AP_50, AP_75, mAP

In [8]:
def compare_result(pt_json1, pt_json2, gt_dir):
    with open(pt_json1, "r") as f:
        pt_dicts1 = json.loads(json.load(f))
    
    with open(pt_json2, "r") as f:
        pt_dicts2 = json.loads(json.load(f))

    n = 1
    oks_all = []
        
    for img_name in pt_dicts1.keys():
            
        pt_kpts1 = read_from_pt_dict(pt_dicts1[img_name])
        pt_kpts2 = read_from_pt_dict(pt_dicts2[img_name])

        json_name = img_name[:-3] + 'json'
        json_file = gt_dir + json_name
        with open(json_file, "r") as f:
            gt_dict = json.load(f)

        gt_kpts = read_from_gt_dict(gt_dict)

        oks1 = evaluate_oks(pt_kpts1, gt_kpts)
        oks2 = evaluate_oks(pt_kpts2, gt_kpts)
        n += 1
        
        for i in range(len(oks1)):
            oks1_single= oks1[i]
            oks2_single= oks2[i]
            if abs(oks1_single - oks2_single) >= 0.3:
                print(img_name)
                print('oks1:', oks1_single)
                print('oks2:', oks2_single)

    return 0

In [13]:
AP_50, AP_75, mAP = evaluate_model("./results_full/mobilenet_model_COCO+RGB_extra//NMS_v2/record.json", './self_dataset/labelme/gt_json_train/')

print("AP_50: ", AP_50)
print("AP_75: ", AP_75)
print("mAP: ", mAP)

Processing img. 1  : left_Explorer_HD720_SN15385_04-00-39.png
0.3415378018973021
Processing img. 2  : left_Explorer_HD720_SN15385_05-00-63.png
Processing img. 3  : left_Explorer_HD720_SN15385_05-00-52.png
0.1657555690608267
Processing img. 4  : left_Explorer_HD720_SN15385_04-00-43.png
0.35070053836441634
Processing img. 5  : left_Explorer_HD720_SN15385_03-00-28.png
Processing img. 6  : left_Explorer_HD720_SN15385_19-31-08.png
Processing img. 7  : left_Explorer_HD720_SN15385_04-00-58.png
0.2510406259318808
Processing img. 8  : left_Explorer_HD720_SN15385_04-00-67.png
0.4746307292658009
Processing img. 9  : left_Explorer_HD720_SN15385_03-00-39.png
Processing img. 10  : left_Explorer_HD720_SN15385_04-00-16.png
Processing img. 11  : left_Explorer_HD720_SN15385_04-00-89.png
Processing img. 12  : left_Explorer_HD720_SN15385_19-31-10.png
Processing img. 13  : left_Explorer_HD720_SN15385_03-00-25.png
0.15139033955171957
Processing img. 14  : left_Explorer_HD720_SN15385_03-00-05.png
Processing 

In [14]:
AP_50, AP_75, mAP = evaluate_model("./results_full/mobilenet_model_COCO_368x765//NMS_v2/record.json", './self_dataset/labelme/gt_json_train/')

print("AP_50: ", AP_50)
print("AP_75: ", AP_75)
print("mAP: ", mAP)

Processing img. 1  : left_Explorer_HD720_SN15385_04-00-39.png
0.19833599578051928
Processing img. 2  : left_Explorer_HD720_SN15385_05-00-63.png
Processing img. 3  : left_Explorer_HD720_SN15385_05-00-52.png
0.14339687150970462
Processing img. 4  : left_Explorer_HD720_SN15385_04-00-43.png
0.1684127838084165
Processing img. 5  : left_Explorer_HD720_SN15385_03-00-28.png
0.27877264401524143
Processing img. 6  : left_Explorer_HD720_SN15385_19-31-08.png
Processing img. 7  : left_Explorer_HD720_SN15385_04-00-58.png
0.12512186799332567
Processing img. 8  : left_Explorer_HD720_SN15385_04-00-67.png
Processing img. 9  : left_Explorer_HD720_SN15385_03-00-39.png
Processing img. 10  : left_Explorer_HD720_SN15385_04-00-16.png
0.37449966257484446
Processing img. 11  : left_Explorer_HD720_SN15385_04-00-89.png
Processing img. 12  : left_Explorer_HD720_SN15385_19-31-10.png
Processing img. 13  : left_Explorer_HD720_SN15385_03-00-25.png
0.267492050287332
Processing img. 14  : left_Explorer_HD720_SN15385_03-